In [1]:
import cv2
import face_recognition
import pickle

filename = 'known_faces.pkl'

with open(filename, 'rb') as file:
    known_faces = pickle.load(file)

known_face_encodings = []
known_face_names = []

for name, encodings in known_faces.items():
    for encoding in encodings:
        known_face_encodings.append(encoding)
        known_face_names.append(name)  # Correctly associate each encoding with the corresponding name

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

print(known_face_encodings)
print(known_face_names)

cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        
        if process_this_frame:
            face_locations = face_recognition.face_locations(small_frame)
            face_encodings = face_recognition.face_encodings(small_frame, face_locations)
            
            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)
                name = "Unknown"
                
                if True in matches:
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]  # Pull the correct name
                
                face_names.append(name)
        
        process_this_frame = not process_this_frame
        
        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 1)
        
        cv2.imshow('Video', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

[array([-0.13520661,  0.09770702,  0.06713311, -0.04410289, -0.13884625,
        0.00598828, -0.06684005, -0.06686851,  0.0391409 , -0.03529978,
        0.21191287, -0.08296858, -0.29483378,  0.0206354 ,  0.04645484,
        0.13663206, -0.15184283, -0.09654125, -0.23463531, -0.08679189,
       -0.03525666, -0.01676514,  0.04487212, -0.05020168, -0.13360611,
       -0.28668454, -0.06130708, -0.11726996, -0.02195128, -0.10758023,
        0.10432144,  0.01519883, -0.21946281, -0.09390845, -0.02620611,
        0.02466463, -0.03580382, -0.03971783,  0.10066675, -0.00358881,
       -0.20861959,  0.06092943,  0.01196875,  0.22377056,  0.28467208,
        0.04247569,  0.0122796 , -0.09877264,  0.05302846, -0.25510758,
        0.0182543 ,  0.09402199,  0.15141416,  0.05555775,  0.03902596,
       -0.01355218,  0.04524967,  0.19727927, -0.20792213,  0.04233339,
        0.08900441, -0.08476054, -0.003399  , -0.09951048,  0.12829252,
        0.11250116, -0.04417515, -0.11827466,  0.21333957, -0.1

2023-10-19 23:05:37.342 Python[10338:385794] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


KeyboardInterrupt: 